In [4]:
from google.colab import files
uploaded = files.upload()


Saving Q1.zip to Q1.zip


In [5]:
!unzip Q1.zip -d /content


Archive:  Q1.zip
   creating: /content/Q1/
   creating: /content/Q1/test/
   creating: /content/Q1/test/Cat/
  inflating: /content/Q1/test/Cat/Cat (1).jpg  
  inflating: /content/Q1/test/Cat/Cat (2).jpg  
  inflating: /content/Q1/test/Cat/Cat (3).jpg  
  inflating: /content/Q1/test/Cat/Cat (4).jpg  
  inflating: /content/Q1/test/Cat/Cat (5).jpg  
   creating: /content/Q1/test/Dog/
  inflating: /content/Q1/test/Dog/Dog (1).jpg  
  inflating: /content/Q1/test/Dog/Dog (2).jpg  
  inflating: /content/Q1/test/Dog/Dog (3).jpg  
  inflating: /content/Q1/test/Dog/Dog (4).jpg  
  inflating: /content/Q1/test/Dog/Dog (5).jpg  
   creating: /content/Q1/train/
   creating: /content/Q1/train/Cat/
  inflating: /content/Q1/train/Cat/cat.0.jpg  
  inflating: /content/Q1/train/Cat/cat.1.jpg  
  inflating: /content/Q1/train/Cat/cat.10.jpg  
  inflating: /content/Q1/train/Cat/cat.100.jpg  
  inflating: /content/Q1/train/Cat/cat.101.jpg  
  inflating: /content/Q1/train/Cat/cat.102.jpg  
  inflating: /conte

In [6]:
!ls /content/Q1


test  train


In [7]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models

BASE_DIR = "/content/Q1"
TRAIN_DIR = f"{BASE_DIR}/train"
TEST_DIR = f"{BASE_DIR}/test"

IMG_SIZE = (150, 150)
BATCH_SIZE = 32
VAL_SPLIT = 0.2
SEED = 42


In [8]:
def load_data():
    train_raw = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        validation_split=VAL_SPLIT,
        subset="training",
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )

    val_raw = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DIR,
        validation_split=VAL_SPLIT,
        subset="validation",
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )

    class_names = train_raw.class_names

    AUTOTUNE = tf.data.AUTOTUNE
    train = train_raw.cache().shuffle(1000).prefetch(AUTOTUNE)
    val = val_raw.cache().prefetch(AUTOTUNE)

    return train, val, class_names

train_ds, val_ds, class_names = load_data()
num_classes = len(class_names)

num_classes, class_names


Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 2000 files belonging to 2 classes.
Using 400 files for validation.


(2, ['Cat', 'Dog'])

In [10]:
def build_simple_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Rescaling(1./255, input_shape=input_shape),

        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),

        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [11]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

def build_vgg16(input_shape, num_classes):
    base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = preprocess_input(inputs)
    x = base(x, training=False)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [12]:
input_shape = IMG_SIZE + (3,)

simple_model = build_simple_cnn(input_shape, num_classes)

print("Training Simple CNN...")
hist_cnn = simple_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training Simple CNN...
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 12s 80ms/step - accuracy: 0.5221 - loss: 0.9077 - val_accuracy: 0.4500 - val_loss: 0.6934
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5313 - loss: 0.6923 - val_accuracy: 0.5525 - val_loss: 0.6814
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5584 - loss: 0.6805 - val_accuracy: 0.5975 - val_loss: 0.6414
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6460 - loss: 0.6193 - val_accuracy: 0.6550 - val_loss: 0.6520
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6792 - loss: 0.5956 - val_accuracy: 0.5975 - val_loss: 0.7089
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7316 - loss: 0.5291 - val_accuracy: 0.6775 - val_loss: 0.6190
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7723 - loss: 0.4575 - val_accuracy: 0.6475 - val_loss: 0.6772
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8306 - loss: 0.3794 - 

In [13]:
input_shape = IMG_SIZE + (3,)

simple_model = build_simple_cnn(input_shape, num_classes)

print("Training Simple CNN...")
hist_cnn = simple_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Training Simple CNN...
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.4861 - loss: 0.7304 - val_accuracy: 0.5825 - val_loss: 0.6508
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5661 - loss: 0.6819 - val_accuracy: 0.6150 - val_loss: 0.6195
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6357 - loss: 0.6448 - val_accuracy: 0.6875 - val_loss: 0.5533
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7107 - loss: 0.5685 - val_accuracy: 0.7100 - val_loss: 0.5317
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7462 - loss: 0.5388 - val_accuracy: 0.7125 - val_loss: 0.5018
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7838 - loss: 0.4337 - val_accuracy: 0.7500 - val_loss: 0.4876
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8361 - loss: 0.4138 - val_accuracy: 0.7275 - val_loss: 0.5289
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8587 - loss: 0.3453 - v

In [14]:
vgg_model = build_vgg16(input_shape, num_classes)

print("Training VGG16 Transfer Learning...")
hist_vgg = vgg_model.fit(
    train_ds.map(lambda x, y: (preprocess_input(x), y)),
    validation_data=val_ds.map(lambda x, y: (preprocess_input(x), y)),
    epochs=10
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training VGG16 Transfer Learning...
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 225ms/step - accuracy: 0.7351 - loss: 10.5270 - val_accuracy: 0.9475 - val_loss: 1.0399
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 128ms/step - accuracy: 0.9383 - loss: 1.7033 - val_accuracy: 0.9525 - val_loss: 0.6548
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - accuracy: 0.9617 - loss: 0.6897 - val_accuracy: 0.9525 - val_loss: 0.6017
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 132ms/step - accuracy: 0.9691 - loss: 0.4754 - val_accuracy: 0.9650 - val_loss: 0.4943
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 132ms/step - accuracy: 0.9840 - loss: 0.1982 - val_accuracy: 0.9575 - val_loss: 0.5150
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 133ms/step - accuracy: 0.9865 - loss: 0.2257 - val_accuracy: 0.9475 - val_loss: 0.8165
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 130ms/step - accuracy: 0.9829 - loss: 0.2260 - val_accuracy: 0.9475 - val_loss: 0.8402
Epoch 8/10
50/5

In [15]:
cnn_best = max(hist_cnn.history["val_accuracy"])
vgg_best = max(hist_vgg.history["val_accuracy"])

cnn_best, vgg_best


(0.75, 0.9700000286102295)

In [16]:
BEST_DIR = "/content/best_model"
os.makedirs(BEST_DIR, exist_ok=True)

best_model = vgg_model  # VGG16 is the best
SAVE_PATH = f"{BEST_DIR}/cats_dogs_best.keras"

best_model.save(SAVE_PATH)

print("Best model saved to:", SAVE_PATH)


Best model saved to: /content/best_model/cats_dogs_best.keras


In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image as keras_image
from google.colab import files
from tensorflow.keras.applications.vgg16 import preprocess_input

MODEL_PATH = "/content/best_model/cats_dogs_best.keras"
IMG_SIZE = (150, 150)

model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded!")

class_names = ["Cat", "Dog"]

def predict_image(img_path):
    img = keras_image.load_img(img_path, target_size=IMG_SIZE)
    arr = keras_image.img_to_array(img)
    arr = preprocess_input(arr)
    arr = np.expand_dims(arr, 0)

    preds = model.predict(arr)[0]
    idx = preds.argmax()
    conf = preds[idx]

    print("\nImage:", img_path)
    print(f"Prediction: {class_names[idx]} (Confidence: {conf:.3f})")

def upload_and_predict():
    uploaded = files.upload()
    for name in uploaded:
        predict_image(name)


Model loaded!


In [18]:
upload_and_predict()


Saving Dog (4).jpg to Dog (4).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Image: Dog (4).jpg
Prediction: Dog (Confidence: 1.000)


In [20]:
upload_and_predict()


Saving Cat (1).jpg to Cat (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

Image: Cat (1).jpg
Prediction: Cat (Confidence: 1.000)


In [21]:
upload_and_predict()


Saving Dog (2).jpg to Dog (2).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step

Image: Dog (2).jpg
Prediction: Dog (Confidence: 1.000)


In [22]:
upload_and_predict()


Saving Cat (5).jpg to Cat (5).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

Image: Cat (5).jpg
Prediction: Cat (Confidence: 1.000)
